# Solar Wind Hydro


#### 1 EXTRACTION LAT LONG / CITIES / WEATHER DATA needed for analysis

* Execute this sequentially in 1st place to get dataset

In [209]:
# Installation to get nearest cities sending coordinates
# pip install citipy

In [52]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
import gmaps
import os
import json



# Import API key
from config import weather_api_key
from config import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes for the whole earth. 
# Lat +90 -90 degrees from Equator to North and South Pole
# longitude East West from Greenwich meridian + - 180 degrees
lat_range = (-90, 90)
lng_range = (-180, 180)

# NOTE. When I commit and push to send my work to GitHub, both Google and WeatherMap 
# sent me mails telling that my API Keys are blocked due to the fact that sending them in 
# config.py or any other folder make them public and can be compromised

# print(weather_api_key)
# print(g_key)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations.
# put size of 1300 to get over 500 cities but do less accesses than default 1500
lats = np.random.uniform(lat_range[0], lat_range[1], size=4)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=4)
# create a tuple with both latitudes and longitudes
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination, use citipy, had to install first
# pip install citipy
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is not already in our list, then add it to cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
# print(cities)

4

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# OpenWeatherMap API Key
api_key = weather_api_key
# Starting URL for Weather Map API Call adding api_key.
# I have decided to extract data in METRIC units instead of IMPERIAL because I understand them better 
# and I am more used to using them
url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=" + api_key
# url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

# Create empty lists to append later API data into lists 
city_list = []
country_list = []
lat_list = []
lng_list = []
temperature_list = []
cloud_list = []
humidity_list = []
wind_list = []

#counter for records
record_counter = 0

print('------------------------')
print('Beginning Weather Check')
print('------------------------')

# begin loop from cities list
for city in cities:
    # Try statement to append calls where value is found 
    # Not all calls return data as OpenWeatherMap will
    # not have have records in all the cities generated by CityPy module
    try: 
        # previous url already with API_key + city to enter in Weather web site 
        query_url = url + "&q=" + city
        # Get weather data
        response = requests.get(query_url).json()

    #   Filling list with data required in next exercises apart from city, lat, long,
    #   humidity, clouds, wind, etc
        city_list.append(response["name"])
        country_list.append(response["sys"]["country"])
        lat_list.append(response["coord"]["lat"])
        lng_list.append(response["coord"]["lon"])
        temperature_list.append(response['main']['temp'])    
        cloud_list.append(response["clouds"]["all"])
        humidity_list.append(response["main"]["humidity"])
        wind_list.append(response["wind"]["speed"])
        
        # Add one to record counter to print it
        record_counter += 1
        print('Retrieving data from city of {} ==> Record {}'.format(city, record_counter))
    
    # If no record found "skip" to next city
    except:
        print('-------------------------')
        print("There are not enough weather information{}. Skip record.")
        print('-------------------------')
    # jump to next record in city list
        pass
record_counter
# ending process after finishing loop
print('-------------------------')
print('Finished weather check')
print('-------------------------')

------------------------
Beginning Weather Check
------------------------
Retrieving data from city of jamestown ==> Record 1
Retrieving data from city of artesia ==> Record 2
Retrieving data from city of leningradskiy ==> Record 3
Retrieving data from city of biak ==> Record 4
-------------------------
Finished weather check
-------------------------


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [4]:
# move data to dictionary and the DF
weather_data_dict = {
    "City": city_list,
    "Lat":lat_list, 
    "Lng":lng_list, 
    "Country":country_list,
    "Temp": temperature_list,    
    "Humidity": humidity_list,   
    "Cloudiness":cloud_list, 
    "Wind":wind_list}

columns = ["City",
           "Lat", 
           "Lng", 
           "Country",
           "Temp",    
           "Humidity",   
           "Cloudiness", 
           "Wind"]

# Create a data frame from dictionary
weather_data_df = pd.DataFrame(weather_data_dict)
#  only getting cities from USA
weather_data_US_df = weather_data_df.loc[weather_data_df["Country"] == "US", columns]
weather_data_US_df = weather_data_US_df.reset_index()
weather_data_US_df = weather_data_US_df.drop(weather_data_US_df.columns[0], axis=1)
# df_after_dropping = df.drop(df.columns[cols],axis=1)
weather_data_US_df



,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind
0,Jamestown,42.0970,-79.2353,US,4.70,93,90,7.21
1,Artesia,33.8658,-118.0831,US,17.78,45,1,4.63


In [5]:
# Display count of weather data values 
# weather_data_df.count()

In [6]:
# Save data frame to CSV
# appending till get 500 US cities
# weather_data_US_df.to_csv('weather_data_US.csv', mode='a', header=True)
# without passing header 
# weather_data_US_df.to_csv('weather_data_US.csv', mode='a', header=False)

# Create Dataframe from csv
working_weather_data_df = pd.read_csv("weather_data_US.csv")
working_weather_data_df

# Drop 1st column not needed
working_weather_data_df = working_weather_data_df.drop(working_weather_data_df.columns[0], axis=1)
working_weather_data_df

# Drop rows with NaN values, in case they exist
# working_weather_data_df = working_weather_data_df.dropna()
working_weather_data_df

# Drop rows with repeated cities
working_weather_data_df = working_weather_data_df.drop_duplicates(subset='City', keep="first")
working_weather_data_df

# Reset index
working_weather_data_df = working_weather_data_df.reset_index()
working_weather_data_df

# Drop 1st column not needed
working_weather_data_df = working_weather_data_df.drop(working_weather_data_df.columns[0], axis=1)
working_weather_data_df


,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind
0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20
1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57
2,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57
3,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04
4,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66
...,...,...,...,...,...,...,...,...
278,Morris,40.8334,-74.5329,US,5.00,86,1,0.44
279,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56
280,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57
281,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57


In [7]:
# Create csv from clean data to store
# it contains 283 cities which accounts for a big sample of almost 10% of every city in USA > 500 unhabitants
working_weather_data_df.to_csv('data/working_weather_data_US.csv')
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind
0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20
1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57
2,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57
3,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04
4,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66
...,...,...,...,...,...,...,...,...
278,Morris,40.8334,-74.5329,US,5.00,86,1,0.44
279,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56
280,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57
281,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57


In [9]:
# fill locations with latitude and longitude to feed markers for hotels
locations = working_weather_data_df[["Lat", "Lng"]]
# locations = working_weather_data_df[["Lat", "Lng"]].astype(float)

#---------------------------------------------------------------------------------------------------------------------
# NOTE: Do not change any of the code in this cell
# Using the template add the city marks to the map
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
city_info = [info_box_template.format(**row) for index, row in working_weather_data_df.iterrows()]


#----------------------------------------------------------------------------------------------------------
# Uncomment following 9 lines if you want to plot a new map without heat layer
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
# plot map with parameters above
fig = gmaps.figure(layout=figure_layout)

# Add marker layer and info box content on top of map
markers = gmaps.marker_layer(locations, info_box_content = city_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

In [11]:
wind = working_weather_data_df["Wind"].astype(float)

# Plot map with center in specific parameters and zoom
# fig = gmaps.figure(center=(40.0, -20.0), zoom_level=2)

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
# plot map with parameters above
fig = gmaps.figure(layout=figure_layout)

# humidity.dtypes
# parameter for additional layer with heat plots, located in previously stored lat and long
# to locations and intensity depending humidity
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=wind, 
                                 dissipating=False, 
                                 max_intensity=15,
                                 point_radius = 2)

# plot heat layer over map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

In [12]:
working_weather_data_df["Altitude (m)"] = ""

# create a params dict that will be updated with new city each iteration
params = {"key": g_key}
# params

# Loop through the working_weather_data_df and run an altitude search for each city
for index, row in working_weather_data_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["locations"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/elevation/json"

    # make request and print url
    altitude = requests.get(base_url, params=params)
  
    # convert to json
    altitude = altitude.json()

    working_weather_data_df.loc[index, "Altitude (m)"] = altitude["results"][0]["elevation"]

# Visualize to confirm altitude appears
# working_weather_data_df.head()

working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m)
0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20,3.048
1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.524
2,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.67
3,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865
4,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66,14.0657
...,...,...,...,...,...,...,...,...,...
278,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.962
279,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.41
280,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.27809
281,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.0792


In [253]:
# working_weather_data_df = working_weather_data_df.to_csv('working_weather_data_alt_US.csv')
# working_weather_data_df

In [13]:
working_weather_data_df = pd.read_csv("data/working_weather_data_alt_US.csv")
working_weather_data_df

,Unnamed: 0,Unnamed: 0.1,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m)
0,0,0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20,3.048000
1,1,1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598
2,2,2,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504
3,3,3,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387
4,4,4,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66,14.065698
...,...,...,...,...,...,...,...,...,...,...,...
278,278,278,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670
279,279,279,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273
280,280,280,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089
281,281,281,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237


In [14]:
gmaps.configure(api_key=g_key)
# print()

altitude = working_weather_data_df["Altitude (m)"].astype(float)
# altitude_df

# # Store 'Latitude' and 'Longitude' into  locations. 
locations = working_weather_data_df[["Lat", "Lng"]].astype(float)

# create map
fig = gmaps.figure()

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
# plot map with parameters above
fig = gmaps.figure(layout=figure_layout)

# humidity.dtypes
# parameter for additional layer with heat plots, located in previously stored lat and long
# to locations and intensity depending ALTITUDE
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=altitude, 
                                 dissipating=True, 
                                 max_intensity=1000,
                                 point_radius = 10)

# plot heat layer over map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

In [17]:
# Drop 1st column not needed
# working_weather_data_df = working_weather_data_df.drop(working_weather_data_df.columns[0], axis=1)
# working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m)
0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20,3.048000
1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598
2,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504
3,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387
4,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66,14.065698
...,...,...,...,...,...,...,...,...,...
278,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670
279,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273
280,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089
281,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237


In [18]:
states_df = pd.read_csv("resources/uscities.csv")
states_df.head()


,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,EST,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,PST,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,CST,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,EST,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,CST,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [19]:
states_df.columns

Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [20]:
states_df = states_df.drop(columns = ['city_ascii', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'ranking', 'zips', 'id'])
states_df

# states_df = states_df.drop(columns = ['state_name'])
states_df
# df_after_dropping = df.drop(df.columns[cols],axis=1)
# df.drop(columns=['B', 'C'])

,city,state_id,state_name,timezone
0,New York,NY,New York,EST
1,Los Angeles,CA,California,PST
2,Chicago,IL,Illinois,CST
3,Miami,FL,Florida,EST
4,Dallas,TX,Texas,CST
...,...,...,...,...
28333,Gross,NE,Nebraska,CST
28334,Lotsee,OK,Oklahoma,CST
28335,The Ranch,MN,Minnesota,CST
28336,Shamrock,OK,Oklahoma,CST


In [21]:
states_df = states_df.rename(columns={"city": "City"})
states_df

,City,state_id,state_name,timezone
0,New York,NY,New York,EST
1,Los Angeles,CA,California,PST
2,Chicago,IL,Illinois,CST
3,Miami,FL,Florida,EST
4,Dallas,TX,Texas,CST
...,...,...,...,...
28333,Gross,NE,Nebraska,CST
28334,Lotsee,OK,Oklahoma,CST
28335,The Ranch,MN,Minnesota,CST
28336,Shamrock,OK,Oklahoma,CST


In [22]:
working_weather_data_df = pd.merge(working_weather_data_df, states_df, how='left', on='City')
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,state_name,timezone
0,Barrow,71.2906,-156.7887,US,-22.0,71,1,7.20,3.048000,NaN,NaN,NaN
1,Palmer,42.1584,-72.3287,US,10.0,32,20,2.57,105.523598,MA,Massachusetts,EST
2,Palmer,42.1584,-72.3287,US,10.0,32,20,2.57,105.523598,AK,Alaska,AKST
3,Palmer,42.1584,-72.3287,US,10.0,32,20,2.57,105.523598,TX,Texas,CST
4,Palmer,42.1584,-72.3287,US,10.0,32,20,2.57,105.523598,PR,Puerto Rico,AST
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,Bloomingdale,27.8936,-82.2404,US,15.0,88,90,2.57,16.079237,IN,Indiana,EST
1015,Bloomingdale,27.8936,-82.2404,US,15.0,88,90,2.57,16.079237,OH,Ohio,EST
1016,Alpena,45.0617,-83.4327,US,8.0,39,1,5.14,179.107376,MI,Michigan,EST
1017,Alpena,45.0617,-83.4327,US,8.0,39,1,5.14,179.107376,AR,Arkansas,CST


In [23]:
# Drop rows with repeated cities
working_weather_data_df = working_weather_data_df.drop_duplicates(subset='City', keep="first")
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,state_name,timezone
0,Barrow,71.2906,-156.7887,US,-22.00,71,1,7.20,3.048000,NaN,NaN,NaN
1,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST
11,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST
12,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST
27,Saint Paul Harbor,57.7900,-152.4072,US,6.00,56,1,5.66,14.065698,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
999,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST
1003,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST
1004,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST
1008,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST


In [25]:
# Drop rows with NaN values, in case they exist
working_weather_data_df = working_weather_data_df.dropna()
working_weather_data_df

# # Drop rows with repeated cities
working_weather_data_df = working_weather_data_df.drop_duplicates(subset='City', keep="first")
working_weather_data_df

# # Reset index
working_weather_data_df = working_weather_data_df.reset_index(drop=True)
working_weather_data_df



,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,state_name,timezone
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST
...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST


In [31]:
working_weather_data_df = working_weather_data_df.to_csv('data/working_weather_data_alt_with_state_US.csv', index=False)
# working_weather_data_df

# Chi-square.
----
Try to see validity of our samples and data

In [32]:
path = "data/working_weather_data_alt_with_state_US.csv"


# Read the mouse data and the study results
working_weather_data_df = pd.read_csv(path)
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,state_name,timezone
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST
...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST


In [33]:
# Get previous data frame and create a new one comparing humidity values in column
samples_per_timezone = working_weather_data_df.groupby(['timezone'])
samples_per_timezone.head()

# # Count number of different timezones previously grouped nunique 
count_samples = samples_per_timezone.count()
count_samples

# Drop not needed columns
count_samples = count_samples.drop(columns = ['Lat', 
                                              'Lng',
                                              'Country',
                                              'Temp',
                                              'Humidity',
                                              'Cloudiness',
                                              'Wind',
                                              'Altitude (m)',
                                              'state_id',
                                              'state_name'])
# count_samples

# Drop not needed rows because Alaska and Hawaii are not significative
count_samples = count_samples.drop(index="AKST")
count_samples = count_samples.drop(index="HST")
# count_samples


# Add a column whose default values are the expected values
count_samples[1] = 61
count_samples

# Rename columns
count_samples.columns = ["observed", "expected"]
count_samples

,observed,expected
timezone,,
CST,79,61
EST,80,61
MST,50,61
PST,45,61


In [34]:
# The degree of freedom is 4-1 = 3
# With a p-value of 0.05, the confidence level is 1.00-0.05 = 0.95.
critical_value = st.chi2.ppf(q = 0.95, df = 3)
critical_value

7.814727903251179

In [35]:
# Run the chi square test with stats.chisquare()
st.chisquare(count_samples['observed'], count_samples['expected'])

Power_divergenceResult(statistic=17.40983606557377, pvalue=0.0005819997882679839)

### Conclusion

* Since the chi square value of 17.40 exceeds the critical value of 7.81, we conclude that the results are statistically significant.

# By State data

In [36]:
# Study data files
path = "resources/Average Annual Sunshine by State.csv"


# Read the mouse data and the study results
sun_df = pd.read_csv(path)
# sun_df

sun_df = sun_df.drop(columns=['% Sun', 'Clear Days'])
sun_df = sun_df.rename(columns={"Total Hours": "Sun Radiation"})

sun_df.head()

,State,Sun Radiation
0,Alabama,2641
1,Alaska,2061
2,Arizona,3806
3,Arkansas,2771
4,California,3055


In [37]:
# Study data files
path = "resources/Precipitation by state.csv"


# Read the mouse data and the study results
precipitation_df = pd.read_csv(path)
precipitation_df.head()

,State,Precip_inch,Precip_mm
0,Alabama,58.3,1480
1,Alaska,22.5,572
2,Arizona,13.6,345
3,Arkansas,50.6,1284
4,California,22.2,563


In [38]:
# Study data files
path = "resources/census Average elevation by state.csv"


# Read the mouse data and the study results
elevation_df = pd.read_csv(path)
elevation_df.head()

,State,mean elevation feet,mean elevation meters
0,Alabama,500,153
1,Alaska,"1,900",580
2,Arizona,"4,100","1,251"
3,Arkansas,650,198
4,California,"2,900",885


In [39]:
# Study data files
path = "resources/State-by State Renewable Energy Consumption.csv"


# Read the mouse data and the study results
# Energy Consumption measured in Trillions of BTU
renewable_cons_df = pd.read_csv(path)
renewable_cons_df.head()

renewable_cons_df = renewable_cons_df[['State','Hydroelectric', 'Solar',
                                       'Wind']]
renewable_cons_df

,State,Hydroelectric,Solar,Wind
0,Alabama,101.4,3.4,0.0
1,Alaska,15.2,0.0,1.4
2,Arizona,63.6,70.8,4.8
3,Arkansas,27.4,2.1,0.0
4,California,239.7,381.7,127.7
5,Colorado,16.6,15.3,88.7
6,Connecticut,5.1,6.5,0.1
7,DC,0.0,0.7,0.0
8,Delaware,0.0,1.5,0.0
9,Florida,2.1,53.5,0.0


In [40]:
states_weather_data_df = pd.merge(sun_df, precipitation_df, how='left', on='State')
states_weather_data_df.head()

,State,Sun Radiation,Precip_inch,Precip_mm
0,Alabama,2641,58.3,1480
1,Alaska,2061,22.5,572
2,Arizona,3806,13.6,345
3,Arkansas,2771,50.6,1284
4,California,3055,22.2,563


In [41]:
states_weather_data_df = pd.merge(states_weather_data_df, elevation_df, how='left', on='State')
states_weather_data_df.head()

,State,Sun Radiation,Precip_inch,Precip_mm,mean elevation feet,mean elevation meters
0,Alabama,2641,58.3,1480,500,153
1,Alaska,2061,22.5,572,"1,900",580
2,Arizona,3806,13.6,345,"4,100","1,251"
3,Arkansas,2771,50.6,1284,650,198
4,California,3055,22.2,563,"2,900",885


In [42]:
states_weather_data_df = pd.merge(states_weather_data_df, renewable_cons_df, how='left', on='State')
states_weather_data_df.head()

,State,Sun Radiation,Precip_inch,Precip_mm,mean elevation feet,mean elevation meters,Hydroelectric,Solar,Wind
0,Alabama,2641,58.3,1480,500,153,101.4,3.4,0.0
1,Alaska,2061,22.5,572,"1,900",580,15.2,0.0,1.4
2,Arizona,3806,13.6,345,"4,100","1,251",63.6,70.8,4.8
3,Arkansas,2771,50.6,1284,650,198,27.4,2.1,0.0
4,California,3055,22.2,563,"2,900",885,239.7,381.7,127.7


# Add state data to previous dataframe

In [43]:
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,state_name,timezone
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST
...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST


In [46]:
# # Rename columns to avoid conflicts
working_weather_data_df = working_weather_data_df.rename(columns={"state_name": "State"})
working_weather_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind,Altitude (m),state_id,State,timezone
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST
...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST


In [47]:
merged_data_df = pd.merge(working_weather_data_df, states_weather_data_df, how='left', on='State')

merged_data_df


,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Wind_x,Altitude (m),state_id,State,timezone,Sun Radiation,Precip_inch,Precip_mm,mean elevation feet,mean elevation meters,Hydroelectric,Solar,Wind_y
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST,2634.0,47.7,1211.0,500,153,10.3,28.2,2.0
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST,3073.0,12.9,328.0,"6,700","2,044",8.9,0.1,36.9
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST,2120.0,41.8,1062.0,"1,000",305,269.8,17.7,36.4
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST,2711.0,27.3,693.0,"1,200",366,9.6,10.4,97.5
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST,2947.0,20.1,511.0,"2,200",671,57.0,0.0,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST,2567.0,39.2,996.0,600,183,1.3,2.7,108.3
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST,3055.0,22.2,563.0,"2,900",885,239.7,381.7,127.7
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST,2606.0,47.9,1218.0,200,61,0.0,1.2,1.4
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST,2927.0,54.5,1385.0,100,31,2.1,53.5,0.0


In [48]:
# # Rename columns to avoid conflicts
merged_data_df = merged_data_df.rename(columns={"state_name": "State",
                                               'Wind_x': 'Local Wind (kph)',
                                                'Sun Radiation': 'Sun Radiation (h)',
                                                'Hydroelectric': 'Hydro cons (BTU)',
                                                'Solar': 'Solar cons (BTU)',
                                               'Wind_y': 'Wind cons (BTU)'})
merged_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Local Wind (kph),Altitude (m),state_id,State,timezone,Sun Radiation (h),Precip_inch,Precip_mm,mean elevation feet,mean elevation meters,Hydro cons (BTU),Solar cons (BTU),Wind cons (BTU)
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST,2634.0,47.7,1211.0,500,153,10.3,28.2,2.0
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST,3073.0,12.9,328.0,"6,700","2,044",8.9,0.1,36.9
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST,2120.0,41.8,1062.0,"1,000",305,269.8,17.7,36.4
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST,2711.0,27.3,693.0,"1,200",366,9.6,10.4,97.5
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST,2947.0,20.1,511.0,"2,200",671,57.0,0.0,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST,2567.0,39.2,996.0,600,183,1.3,2.7,108.3
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST,3055.0,22.2,563.0,"2,900",885,239.7,381.7,127.7
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST,2606.0,47.9,1218.0,200,61,0.0,1.2,1.4
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST,2927.0,54.5,1385.0,100,31,2.1,53.5,0.0


In [53]:
merged_data_df = merged_data_df.to_csv('data/merged_data_df.csv', index=False)

In [54]:
merged_data_df = pd.read_csv('data/merged_data_df.csv')
# merged_data_df

merged_data_df = merged_data_df.dropna()
# merged_data_df
merged_data_df

,City,Lat,Lng,Country,Temp,Humidity,Cloudiness,Local Wind (kph),Altitude (m),state_id,State,timezone,Sun Radiation (h),Precip_inch,Precip_mm,mean elevation feet,mean elevation meters,Hydro cons (BTU),Solar cons (BTU),Wind cons (BTU)
0,Palmer,42.1584,-72.3287,US,10.00,32,20,2.57,105.523598,MA,Massachusetts,EST,2634.0,47.7,1211.0,500,153,10.3,28.2,2.0
1,Laramie,41.3114,-105.5911,US,-1.00,80,90,2.57,2184.666504,WY,Wyoming,MST,3073.0,12.9,328.0,"6,700","2,044",8.9,0.1,36.9
2,Jamestown,42.0970,-79.2353,US,12.00,16,1,3.04,419.865387,NY,New York,EST,2120.0,41.8,1062.0,"1,000",305,269.8,17.7,36.4
3,Willmar,45.1219,-95.0433,US,8.00,76,1,5.14,349.588837,MN,Minnesota,CST,2711.0,27.3,693.0,"1,200",366,9.6,10.4,97.5
4,Mitchell,43.7094,-98.0298,US,9.00,57,1,3.60,399.903931,SD,South Dakota,CST,2947.0,20.1,511.0,"2,200",671,57.0,0.0,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,US,5.00,86,1,0.44,286.961670,IL,Illinois,CST,2567.0,39.2,996.0,600,183,1.3,2.7,108.3
265,South Lake Tahoe,38.9332,-119.9844,US,3.89,55,1,1.56,1900.405273,CA,California,PST,3055.0,22.2,563.0,"2,900",885,239.7,381.7,127.7
266,Warwick,41.7001,-71.4162,US,5.00,93,1,1.57,5.278089,RI,Rhode Island,EST,2606.0,47.9,1218.0,200,61,0.0,1.2,1.4
267,Bloomingdale,27.8936,-82.2404,US,15.00,88,90,2.57,16.079237,FL,Florida,EST,2927.0,54.5,1385.0,100,31,2.1,53.5,0.0


In [50]:
# generate dataframe from previous one keeping only columns needed for this calculations
reduced_df = merged_data_df[['City','Lat','Lng', 'Local Wind (kph)', 'Altitude (m)', 'state_id', 
                             'State','timezone', 'Sun Radiation (h)', 'Precip_inch', 'Precip_mm',
                            'mean elevation feet', 'mean elevation meters', 'Hydro cons (BTU)',
                            'Solar cons (BTU)', 'Wind cons (BTU)']]


reduced_df

,City,Lat,Lng,Local Wind (kph),Altitude (m),state_id,State,timezone,Sun Radiation (h),Precip_inch,Precip_mm,mean elevation feet,mean elevation meters,Hydro cons (BTU),Solar cons (BTU),Wind cons (BTU)
0,Palmer,42.1584,-72.3287,2.57,105.523598,MA,Massachusetts,EST,2634.0,47.7,1211.0,500,153,10.3,28.2,2.0
1,Laramie,41.3114,-105.5911,2.57,2184.666504,WY,Wyoming,MST,3073.0,12.9,328.0,"6,700","2,044",8.9,0.1,36.9
2,Jamestown,42.0970,-79.2353,3.04,419.865387,NY,New York,EST,2120.0,41.8,1062.0,"1,000",305,269.8,17.7,36.4
3,Willmar,45.1219,-95.0433,5.14,349.588837,MN,Minnesota,CST,2711.0,27.3,693.0,"1,200",366,9.6,10.4,97.5
4,Mitchell,43.7094,-98.0298,3.60,399.903931,SD,South Dakota,CST,2947.0,20.1,511.0,"2,200",671,57.0,0.0,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Morris,40.8334,-74.5329,0.44,286.961670,IL,Illinois,CST,2567.0,39.2,996.0,600,183,1.3,2.7,108.3
265,South Lake Tahoe,38.9332,-119.9844,1.56,1900.405273,CA,California,PST,3055.0,22.2,563.0,"2,900",885,239.7,381.7,127.7
266,Warwick,41.7001,-71.4162,1.57,5.278089,RI,Rhode Island,EST,2606.0,47.9,1218.0,200,61,0.0,1.2,1.4
267,Bloomingdale,27.8936,-82.2404,2.57,16.079237,FL,Florida,EST,2927.0,54.5,1385.0,100,31,2.1,53.5,0.0


In [51]:
final_data_df = reduced_df.to_csv('data/final_data_df.csv', index=False)
# working_weather_data_df